## Постановка задачи: 
разработать систему предсказания оттока клиентов для компании
Т.к. данные достаточно сложно парсить, был выбран готовый датасет с контекста Data Fusion Contest 2024, где предоставлены данные от банка ВТБ,
для удобства скачивания весь датасет был выложен на kaggle



In [1]:
import kaggle
import json


import os
try :
    with open("../kaggle.json", "r") as file:
        kaggle_auth_data = json.load(file)
        os.environ['KAGGLE_USERNAME'] = kaggle_auth_data["username"]
        os.environ['KAGGLE_KEY'] = kaggle_auth_data["key"]
except FileNotFoundError:
    print("no_auth_file_was_found")
kaggle.api.authenticate()
kaggle.api.dataset_download_files('krellbob/churn-prediction-from-data-fusin-2024', path="../data_", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/krellbob/churn-prediction-from-data-fusin-2024


- clients.csv - информация о клиентах 
- transactions.csv - ранзакции клиентов банка 
- reports_dates.csv - информация о датах отчетов 
- train.csv - данные о таргетах  

In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("../data_/train.csv")

In [3]:
train_data.head()

,user_id,target,time
0,3,0,77
1,13,0,86
2,37,0,89
3,41,0,57
4,42,0,84


In [5]:
clients = pd.read_csv('../data_/clients.csv')

In [6]:
clients.head()

,user_id,report,employee_count_nm,bankemplstatus,customer_age
0,3,2,ОТ 101 ДО 500,0,3
1,9,1,БОЛЕЕ 1001,0,3
2,13,6,ОТ 501 ДО 1000,0,2
3,37,5,БОЛЕЕ 1001,0,2
4,41,1,ОТ 101 ДО 500,0,2


In [10]:
report_dates = pd.read_csv("../data_/report_dates.csv", parse_dates=['report_dt'])

In [12]:
transactions = pd.read_csv("../data_/transactions.csv", parse_dates=["transaction_dttm"])

In [13]:
transactions = transactions.sort_values("transaction_dttm").reset_index(drop=True)

In [14]:
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,124092,1,1,-437.722809,2021-10-20 09:00:00
1,198674,7,1,-1421.831177,2021-10-20 09:00:03
2,209115,0,1,46684.066406,2021-10-20 09:00:05
3,447095,1,1,-94.699913,2021-10-20 09:00:18
4,348,8,1,-838.885559,2021-10-20 09:00:35
...,...,...,...,...,...
13075018,510219,4,1,-75.810562,2023-03-20 20:58:54
13075019,490168,1,1,-523.368469,2023-03-20 20:59:03
13075020,27159,79,1,-687.029541,2023-03-20 20:59:16
13075021,311184,155,1,-1358.584229,2023-03-20 20:59:42


In [19]:
sample_data = pd.read_csv("../data_/sample_submit_naive.csv")

In [20]:
sample_data['target'] = -1

In [21]:
train_data = pd.concat([train_data, sample_data])

In [22]:
train_data

,user_id,target,time,predict
0,3,0,77.0,NaN
1,13,0,86.0,NaN
2,37,0,89.0,NaN
3,41,0,57.0,NaN
4,42,0,84.0,NaN
...,...,...,...,...
31995,561362,-1,NaN,-2.438619
31996,561419,-1,NaN,-2.438619
31997,561895,-1,NaN,-2.437339
31998,561908,-1,NaN,-2.437339


In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
train_data = train_data.merge(clients, how="left", on="user_id")

In [25]:
train_data.head()

,user_id,target,time,predict,report,employee_count_nm,bankemplstatus,customer_age
0,3,0,77.0,NaN,2,ОТ 101 ДО 500,0,3
1,13,0,86.0,NaN,6,ОТ 501 ДО 1000,0,2
2,37,0,89.0,NaN,5,БОЛЕЕ 1001,0,2
3,41,0,57.0,NaN,1,ОТ 101 ДО 500,0,2
4,42,0,84.0,NaN,12,ДО 10,0,3


In [27]:
train_data["employee_count_nm"] = LabelEncoder().fit_transform(train_data["employee_count_nm"].fillna("no_data"))

In [28]:
train_data

,user_id,target,time,predict,report,employee_count_nm,bankemplstatus,customer_age
0,3,0,77.0,NaN,2,4,0,3
1,13,0,86.0,NaN,6,8,0,2
2,37,0,89.0,NaN,5,1,0,2
3,41,0,57.0,NaN,1,4,0,2
4,42,0,84.0,NaN,12,3,0,3
...,...,...,...,...,...,...,...,...
95995,561362,-1,NaN,-2.438619,12,0,0,3
95996,561419,-1,NaN,-2.438619,12,0,0,3
95997,561895,-1,NaN,-2.437339,12,0,0,2
95998,561908,-1,NaN,-2.437339,12,0,0,2


In [32]:
codes = transactions["mcc_code"].value_counts()

In [47]:
filtered_codes = [code for code in codes if code >= 15]

In [48]:
print(len(filtered_codes))

281


In [45]:
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,124092,1,1,-437.722809,2021-10-20 09:00:00
1,198674,7,1,-1421.831177,2021-10-20 09:00:03
2,209115,0,1,46684.066406,2021-10-20 09:00:05
3,447095,1,1,-94.699913,2021-10-20 09:00:18
4,348,8,1,-838.885559,2021-10-20 09:00:35
...,...,...,...,...,...
13075018,510219,4,1,-75.810562,2023-03-20 20:58:54
13075019,490168,1,1,-523.368469,2023-03-20 20:59:03
13075020,27159,79,1,-687.029541,2023-03-20 20:59:16
13075021,311184,155,1,-1358.584229,2023-03-20 20:59:42


In [57]:
codes_table = transactions[transactions.mcc_code.isin(filtered_codes)].pivot_table(
    index = "user_id",
    values=["transaction_amt"],
    columns=["mcc_code"],
    aggfunc=["count", "median", "sum"]
).fillna(0)

In [58]:
codes_table

count                                                ...  \
         transaction_amt                                                ...   
mcc_code             15   16    17   18   22   23   26   28   29   31   ...   
user_id                                                                 ...   
3                    0.0  1.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
9                    4.0  0.0   0.0  1.0  0.0  3.0  0.0  0.0  0.0  2.0  ...   
37                   2.0  1.0   0.0  1.0  5.0  0.0  0.0  7.0  0.0  0.0  ...   
41                   1.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
42                   3.0  0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
...                  ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...   
562043               1.0  2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
562205               0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
562312               0.0  0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...   
562721               0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  ...   
562740               0.0  1.0  52.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...   

                     sum                                               
         transaction_amt                                               
mcc_code             246  249  264  266  276  296  318  338  364  412  
user_id                                                                
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
37                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
41                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
42                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
562043               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
562205               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
562312               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
562721               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
562740               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[81085 rows x 201 columns]